# Regression
- **Usage**: The cost function that is minimized to choose split points is the sum squared error across all training samples that fall within the rectangle.
- **Formula**:
$$SSE = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$
- **Example**: Suppose we have a dataset where the target variable is the price of houses based on different features like size, number of rooms, etc. We can use regression to predict the price of a new house by minimizing the sum squared error between the predicted prices and actual prices of training samples.

# Classification
- **Usage**: The Gini cost function is used to provide an indication of how pure the nodes are. Node purity refers to how mixed the training data assigned to each node is.
- **Formula**:
$$Gini = 1 - \sum_{k=1}^{K} p_k^2$$
- **Example**: Suppose we have a dataset of flowers classified into different species based on features like petal length, petal width, etc. The Gini index can be used to evaluate the splits in the dataset and ensure that the nodes are pure, meaning that the flowers in each node are primarily of one species.

## Gini Index
- **Definition**: The Gini index evaluates splits in the dataset.
- **Usage**: It involves one input attribute and its value to divide training patterns into two groups of rows.
- **Gini Score**:
  - Indicates how mixed the classes are in the two groups created by the split.
  - A perfect separation has a Gini score of 0.
  - The worst-case split (50/50 classes in each group) has a Gini score of 1.0 (for a 2 class problem).
- **Formula**:
$$Gini = 1 - \sum_{k=1}^{K} p_k^2$$


# Formulas:
### Proportion:
The formula for the proportion is:
$$Proportion = \frac{count(\text{class value})}{count(\text{rows})}$$
### Gini Index:
The formula for the Gini index is:
$$Gini = \sum_{i=1}^{n} (proportion_i \times (1.0 - proportion_i))$$


###  Build a Tree

Creating the root node of the tree is simple using the `get split()` function on the entire dataset. Adding more nodes involves three main parts:
1. **Terminal Nodes**
2. **Recursive Splitting**
3. **Building a Tree**

#### Terminal Nodes
- **Stopping Criteria**: Use depth and the number of rows the node handles.
- **Maximum Tree Depth**: Stop splitting when the maximum depth is reached to avoid overfitting.
- **Minimum Node Records**: Stop splitting if the node has too few training patterns to avoid overfitting.

### Stopping Conditions
- If all rows belong to one group after a split, stop splitting as there's no data to split further.
- Terminal nodes are used for final predictions by selecting the most common class value among assigned rows.



### Recursive Splitting

Building a decision tree involves calling the `get split()` function repeatedly on the groups created for each node. New nodes added to an existing node are called child nodes. A node may have:
- Zero children (a terminal node)
- One child (one side makes a prediction directly)
- Two child nodes (left and right in the dictionary representation)

Once a node is created, child nodes are created recursively on each group of data from the split by calling the same function again.

#### Recursive Procedure
This function takes a node as an argument along with the maximum depth, the minimum number of patterns in a node, and the current depth of a node. Here's how it works:

1. Extract the two groups of data split by the node for use and delete them from the node.
2. Check if either left or right group of rows is empty; if so, create a terminal node using the available records.
3. Check if the maximum depth is reached; if so, create a terminal node.
4. Process the left child:
   - Create a terminal node if the group of rows is too small.
   - Otherwise, create and add the left node in a depth-first fashion until the bottom of the tree is reached on this branch.
5. Process the right child in the same manner, moving back up the constructed tree to the root.




# UCI Banknote Authentication Dataset

## Overview
The UCI Banknote Authentication dataset is designed to distinguish between authentic (genuine) and inauthentic (forged) banknotes. The dataset contains features extracted from images of banknotes and includes the following attributes:

1. **Variance** of the Wavelet Transformed image
2. **Skewness** of the Wavelet Transformed image
3. **Kurtosis** of the Wavelet Transformed image
4. **Entropy** of the image
5. **Class**: 0 for genuine and 1 for forged

## Features
- **Variance**: Measures the variation in the wavelet transformed image.
- **Skewness**: Measures the asymmetry in the wavelet transformed image.
- **Kurtosis**: Measures the tailedness of the wavelet transformed image.
- **Entropy**: Measures the randomness in the image.

# Importing dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data")

print("Path to dataset files:", path)

100%|██████████| 19.2k/19.2k [00:00<00:00, 19.2MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ritesaluja/bank-note-authentication-uci-data/versions/1


# Code

In [ ]:
from random import seed
from random import randrange
from csv import reader
import os

# Load a CSV file
def load_csv(filename):
    dataset = list()
    # Get a list of all CSV files in the directory
    csv_files = [f for f in os.listdir(filename) if f.endswith('.csv')]
    # If there are CSV files in the directory, use the first one
    if csv_files:
        csv_file_path = os.path.join(filename, csv_files[0])
        with open(csv_file_path, 'r') as file:
            csv_reader = reader(file)
            for row in csv_reader:
                if not row:
                    continue
                dataset.append(row)  # Add each row to the dataset list
        return dataset
    else:
        raise FileNotFoundError("No CSV file found in the directory.")

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset[1:]:  # Skip header row
        row[column] = float(row[column].strip())  # Convert each value in the specified column to float

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset[1:])  # Skip header row
    fold_size = len(dataset_copy) // n_folds  # Ensure fold_size is an integer
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))  # Remove the selected element from dataset_copy and add it to the fold
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)  # Remove the current fold from the train_set
        train_set = sum(train_set, [])  # Flatten the list
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None  # Remove the target value
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]  # Extract the target values
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # Check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

# Test CART on Bank Note dataset
seed(1)
# Load and prepare data
filename = '/root/.cache/kagglehub/datasets/ritesaluja/bank-note-authentication-uci-data/versions/1'
dataset = load_csv(filename)
# Convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# Evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))


Scores: [83.57664233576642, 82.84671532846716, 86.86131386861314, 79.92700729927007, 82.11678832116789]
Mean Accuracy: 83.066%
